In [7]:
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score

class TrainBatcher(object):

    # Class constructor
    def __init__(self, examples, labels):
        self.labels = labels
        self.examples = examples
        self.index_in_epoch = 0
        self.num_examples = examples.shape[0]

    # mini-batching method
    def next_batch(self, batch_size):
        start = self.index_in_epoch
        self.index_in_epoch += batch_size

        # When all the training data is ran, shuffles it
        if self.index_in_epoch > self.num_examples:
            perm = np.arange(self.num_examples)
            np.random.shuffle(perm)
            self.examples = self.examples[perm]
            self.labels = self.labels[perm]
            # Start next epoch
            start = 0
            self.index_in_epoch = batch_size
            assert batch_size <= self.num_examples
        end = self.index_in_epoch
        
        return self.examples[start:end], self.labels[start:end]
###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ######

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
orig_y_test1 = y_test1

X_all = np.concatenate((X_train1, X_valid1), axis=0) 
y_all = np.concatenate((y_train1, y_valid1), axis=0)

fold_index = list(range(X_all.shape[0])) # [0. ,1.,2. , ..., X_all.shape[0]-1]
fold_num = 5

k_fold = KFold(n_splits=fold_num) # split data into 5 folds
folds_accuracy = 0
best_fold_accuracy = 0
best_fold = 0

k = 0
for train_indices, val_indices in k_fold.split(fold_index):
    k += 1
   
    X_train1 = X_all[train_indices]
    y_train1 = y_all[train_indices]
    
    X_valid1 = X_all[val_indices]
    y_valid1 = y_all[val_indices]
    
    print('=====================================')
    print("fold %d:" % k)
    print('-------------------------------------')
    
    enc = OneHotEncoder(sparse=False)
    y_train1 = enc.fit_transform(y_train1.reshape(len(y_train1),1))
    y_valid1 = enc.fit_transform(y_valid1.reshape(len(y_valid1),1))
    y_test1 = enc.fit_transform(orig_y_test1.reshape(len(orig_y_test1),1))    

    learning_rate = 1e-3
    batch_size = 100
    iteration = 100

    sess = tf.InteractiveSession()
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, shape=[None, 5])
    keep_prob = tf.placeholder(tf.float32)
    he_init = tf.contrib.layers.variance_scaling_initializer()

    # d1
    dp1 = tf.nn.dropout(x, keep_prob)
    d1 = tf.layers.dense(dp1,128,activation=tf.nn.elu,kernel_initializer=he_init)
    # d2
    dp2 = tf.nn.dropout(d1,keep_prob)
    d2 = tf.layers.dense(dp2,128,activation=tf.nn.elu,kernel_initializer=he_init)
    # d3
    dp3 = tf.nn.dropout(d2, keep_prob)
    d3 = tf.layers.dense(dp3,128,activation=tf.nn.elu,kernel_initializer=he_init)
    # d4
    dp4 = tf.nn.dropout(d3, keep_prob)
    d4 = tf.layers.dense(dp4,128,activation=tf.nn.elu,kernel_initializer=he_init)
    # d5
    pred = tf.layers.dense(d4,5)


    # Cost Function - We use the softmax cross-entropy as our loss. This is the numerically stable implementation
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=pred))
    train_step  = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    # Evaluate model 
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y_,1)) 
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    fold_saver = tf.train.Saver(tf.global_variables())
    model_saver = tf.train.Saver(tf.global_variables())
    # Start TensorFlow session
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Create a batcher object to provide the data mini-batches to our trainer
        mnist = TrainBatcher(X_train1, y_train1)

        best_train_accuracy = 0
        last_improvement = 0
        earlystopping_itr = 0
        earlystopped = False

        print('Starting training...')
        for i in range(5000):
            batch_xs, batch_ys = mnist.next_batch(batch_size)

            # Logs every 100 iterations
            if i % 100 == 0:   
                #train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0})
                train_accuracy = accuracy.eval(feed_dict={x: X_valid1, y_: y_valid1, keep_prob: 1.0})
                if train_accuracy >= best_train_accuracy:
                    best_train_accuracy = train_accuracy
                    last_improvement = i
                    earlystopping_itr = 0
                    fold_save_path = fold_saver.save(sess, './Team45_HW2_fold.ckpt')
                else: 
                    earlystopping_itr += 1     

                if earlystopping_itr >= 10:
                    earlystopped = True
                    print("No improvement after step %d accuracy %g" % (last_improvement, best_train_accuracy))
                    break

                print("step %d, accuracy %g (best: %g)" % (i, train_accuracy, best_train_accuracy))

            train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.8})
        
        print('Training finished')
        print('-------------------------------------')
        fold_saver.restore(sess, fold_save_path)
        if best_train_accuracy > best_fold_accuracy:
            best_fold = k
            best_fold_accuracy = best_train_accuracy
            model_save_path = model_saver.save(sess, './Team45_HW2.ckpt')
                    
        folds_accuracy += best_train_accuracy
        print('accuracy %g (best fold accuracy: %g)' %(best_train_accuracy, best_fold_accuracy))

fold 1:
-------------------------------------
Starting training...
0.307353
step 0, accuracy 0.307353 (best: 0.307353)
0.960458
step 100, accuracy 0.960458 (best: 0.960458)
0.962255
step 200, accuracy 0.962255 (best: 0.962255)
0.970425
step 300, accuracy 0.970425 (best: 0.970425)
0.974837
step 400, accuracy 0.974837 (best: 0.974837)
0.977778
step 500, accuracy 0.977778 (best: 0.977778)
0.977288
step 600, accuracy 0.977288 (best: 0.977778)
0.981046
step 700, accuracy 0.981046 (best: 0.981046)
0.979902
step 800, accuracy 0.979902 (best: 0.981046)
0.981373
step 900, accuracy 0.981373 (best: 0.981373)
0.983007
step 1000, accuracy 0.983007 (best: 0.983007)
0.981536
step 1100, accuracy 0.981536 (best: 0.983007)
0.981699
step 1200, accuracy 0.981699 (best: 0.983007)
0.982843
step 1300, accuracy 0.982843 (best: 0.983007)
0.983987
step 1400, accuracy 0.983987 (best: 0.983987)
0.986438
step 1500, accuracy 0.986438 (best: 0.986438)
0.98366
step 1600, accuracy 0.98366 (best: 0.986438)
0.984804
ste

INFO:tensorflow:Restoring parameters from ./Team45_HW2_fold.ckpt


accuracy 0.990359 (best fold accuracy: 0.990359)
fold 2:
-------------------------------------
Starting training...
0.188756
step 0, accuracy 0.188756 (best: 0.188756)
0.963229
step 100, accuracy 0.963229 (best: 0.963229)
0.961758
step 200, accuracy 0.961758 (best: 0.963229)
0.975159
step 300, accuracy 0.975159 (best: 0.975159)
0.975813
step 400, accuracy 0.975813 (best: 0.975813)
0.979082
step 500, accuracy 0.979082 (best: 0.979082)
0.980552
step 600, accuracy 0.980552 (best: 0.980552)
0.982677
step 700, accuracy 0.982677 (best: 0.982677)
0.981696
step 800, accuracy 0.981696 (best: 0.982677)
0.983004
step 900, accuracy 0.983004 (best: 0.983004)
0.984148
step 1000, accuracy 0.984148 (best: 0.984148)
0.985292
step 1100, accuracy 0.985292 (best: 0.985292)
0.984148
step 1200, accuracy 0.984148 (best: 0.985292)
0.985782
step 1300, accuracy 0.985782 (best: 0.985782)
0.985945
step 1400, accuracy 0.985945 (best: 0.985945)
0.986599
step 1500, accuracy 0.986599 (best: 0.986599)
0.98758
step 160

INFO:tensorflow:Restoring parameters from ./Team45_HW2_fold.ckpt


accuracy 0.992482 (best fold accuracy: 0.992482)
fold 3:
-------------------------------------
Starting training...
0.144631
step 0, accuracy 0.144631 (best: 0.144631)
0.965354
step 100, accuracy 0.965354 (best: 0.965354)
0.966498
step 200, accuracy 0.966498 (best: 0.966498)
0.973198
step 300, accuracy 0.973198 (best: 0.973198)
0.974996
step 400, accuracy 0.974996 (best: 0.974996)
0.97614
step 500, accuracy 0.97614 (best: 0.97614)
0.979899
step 600, accuracy 0.979899 (best: 0.979899)
0.979082
step 700, accuracy 0.979082 (best: 0.979899)
0.979572
step 800, accuracy 0.979572 (best: 0.979899)
0.980552
step 900, accuracy 0.980552 (best: 0.980552)
0.983167
step 1000, accuracy 0.983167 (best: 0.983167)
0.984965
step 1100, accuracy 0.984965 (best: 0.984965)
0.984638
step 1200, accuracy 0.984638 (best: 0.984965)
0.985128
step 1300, accuracy 0.985128 (best: 0.985128)
0.984475
step 1400, accuracy 0.984475 (best: 0.985128)
0.987089
step 1500, accuracy 0.987089 (best: 0.987089)
0.985619
step 1600,

...

accuracy 0.990848 (best fold accuracy: 0.992482)
fold 4:
-------------------------------------
Starting training...
0.108188
step 0, accuracy 0.108188 (best: 0.108188)
0.959144
step 100, accuracy 0.959144 (best: 0.959144)
0.968949
step 200, accuracy 0.968949 (best: 0.968949)
0.970583
step 300, accuracy 0.970583 (best: 0.970583)
0.975486
step 400, accuracy 0.975486 (best: 0.975486)
0.973688
step 500, accuracy 0.973688 (best: 0.975486)
0.975813
step 600, accuracy 0.975813 (best: 0.975813)
0.980389
step 700, accuracy 0.980389 (best: 0.980389)
0.980716
step 800, accuracy 0.980716 (best: 0.980716)
0.977938
step 900, accuracy 0.977938 (best: 0.980716)
0.979245
step 1000, accuracy 0.979245 (best: 0.980716)
0.985292
step 1100, accuracy 0.985292 (best: 0.985292)
0.979408
step 1200, accuracy 0.979408 (best: 0.985292)
0.980552
step 1300, accuracy 0.980552 (best: 0.985292)
0.983004
step 1400, accuracy 0.983004 (best: 0.985292)
0.983657
step 1500, accuracy 0.983657 (best: 0.985292)
0.982677
step 16

INFO:tensorflow:Restoring parameters from ./Team45_HW2_fold.ckpt


accuracy 0.990031 (best fold accuracy: 0.992482)
fold 5:
-------------------------------------
Starting training...
0.221278
step 0, accuracy 0.221278 (best: 0.221278)
0.96993
step 100, accuracy 0.96993 (best: 0.96993)
0.978591
step 200, accuracy 0.978591 (best: 0.978591)
0.979735
step 300, accuracy 0.979735 (best: 0.979735)
0.982677
step 400, accuracy 0.982677 (best: 0.982677)
0.984148
step 500, accuracy 0.984148 (best: 0.984148)
0.982023
step 600, accuracy 0.982023 (best: 0.984148)
0.985945
step 700, accuracy 0.985945 (best: 0.985945)
0.987089
step 800, accuracy 0.987089 (best: 0.987089)
0.985782
step 900, accuracy 0.985782 (best: 0.987089)
0.987743
step 1000, accuracy 0.987743 (best: 0.987743)
0.990031
step 1100, accuracy 0.990031 (best: 0.990031)
0.987416
step 1200, accuracy 0.987416 (best: 0.990031)
0.989051
step 1300, accuracy 0.989051 (best: 0.990031)
0.989541
step 1400, accuracy 0.989541 (best: 0.990031)
0.989868
step 1500, accuracy 0.989868 (best: 0.990031)
0.989704
step 1600,

INFO:tensorflow:Restoring parameters from ./Team45_HW2_fold.ckpt


accuracy 0.993463 (best fold accuracy: 0.993463)


In [9]:
model_saver = tf.train.Saver(tf.global_variables())
# Start TensorFlow session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model_saver.restore(sess, model_save_path)
    print('best fold %d with accuracy: %g' %(best_fold, best_fold_accuracy))
    print('average fold accuracy: %g' %(folds_accuracy / fold_num))
    print('test accuracy: %g' %accuracy.eval(feed_dict={x: X_test1, y_: y_test1, keep_prob: 1.0}))
    print('=====================================')
    y = tf.constant(orig_y_test1)
    p = tf.argmax(pred, 1)
    
    print('label\tprecision\t recall')
    print('-------------------------------------')
    for i in range(5):
        p_i = tf.where(tf.equal(p, i)) # indexs of all prediction i 
        print(p_i)
        y_i = tf.where(tf.equal(y, i)) # indexs of all true label i 
        print(y_i)
        correct = tf.equal(tf.gather(y, p_i), i) # check if values of true label y[idxs] = i

        TP = tf.reduce_sum(tf.cast(correct, tf.int32)) # count of true num (prediction is equal to true label)
        FP = tf.shape(p_i)[0] - TP
        FN = tf.shape(y_i)[0] - TP

        precision = TP / (TP + FP)  
        recall = TP / (TP + FN)
        
        prediction_txt = sess.run(precision, feed_dict={x: X_test1, y_: y_test1, keep_prob: 1.0})
        recall_txt = sess.run(recall, feed_dict={x: X_test1, y_: y_test1, keep_prob: 1.0})
        print('%d\t%.10f\t%.10f' %(i, prediction_txt,recall_txt))

INFO:tensorflow:Restoring parameters from ./Team45_HW2.ckpt


INFO:tensorflow:Restoring parameters from ./Team45_HW2.ckpt


best fold 5 with accuracy: 0.993463
average fold accuracy: 0.991437
test accuracy: 0.993773
label	precision	 recall
-------------------------------------
0	0.9949135300	0.9979591837
1	0.9973544974	0.9964757709
2	0.9855072464	0.9883720930
3	0.9940417080	0.9910891089
4	0.9969387755	0.9949083503
